### Step 1 - Loads packages required

In [2]:
import pandas as pd
import pandasql as ps

### Step 2 - Interface file exploration to validate separators and text qualifiers

In [5]:
from custom_procedures import newestFile
sourceFolder = '/content/drive/Shareddrives/Esperanto/Supports/Supplier/'
sourceFile = newestFile(sourceFolder)
if sourceFile:
    print(sourceFile)
#import os
#sourceFile = '/content/drive/Shareddrives/Esperanto/Supports/Supplier/'1. SUPPLIERS_INTERFACE_COPPEL.csv''
#if os.path.exists(sourceFile):
#    print('file exits')
